## **Assignment 2 - Systems Biology**
### **Hugo Manuel Alves Henriques e Silva, hugoalv@student.chalmers.se**

#### **How to run:**
`Run the cells below in order. Make sure to have the datasets in the same folder as the notebook.`

## **Question 3**

### **Problem Description & Assumptions**

- We essentially can view this problem as having 2 networks. A network for the healthy cells and a network for the cancer ones.

- The enzymes we can target are the enzymes that are essential for the cancer cells but non essential for the healthy ones. This mean that
even if we target one of these enzymes, the cancer cells will die but the healthy ones will find an alternative way to survive.

- A very important consideration was made for this problem. The enzymes with RNA count equal to zero in each cell are not considered,
i. e., it is as if they do not exist in the respective cells. This is how we obtain the 2 networks which will be subsets of the
original network from the previous question.

#### **Build the original network along with helpful sets**

In [73]:
# import data
import pandas as pd
df = pd.read_csv('RNAcounts.csv')
df_network = pd.read_csv('ccm.csv')


network = {}

for i in range(len(df_network)):

    from_metabolite = df_network.iloc[i]['From']
    to_metabolite = df_network.iloc[i]['To']
    enzyme = df_network.iloc[i]['Enzyme']

    if from_metabolite not in network:
        network[from_metabolite] = [(enzyme, to_metabolite)]
    else:
        network[from_metabolite].append((enzyme, to_metabolite))

print(network)

enzymes = []
for metabolite in network:
    for enzyme in network[metabolite]:
        enzymes.append(enzyme[0])
enzymes = list(set(enzymes))
print(enzymes)

# endpoints are "to" elements in the dataframe without [...] in the end of the string
endpoints = []
for i in range(len(df_network)):
    if df_network.iloc[i]['To'][-1:] != ']':
        endpoints.append(df_network.iloc[i]['To'])
endpoints = list(set(endpoints))
print(endpoints)

{'Glucose [c]': [('Enzyme1', 'Glucose-6-phosphate [c]'), ('Enzyme33', 'ADP-Glucose [c]')], 'Glucose-6-phosphate [c]': [('Enzyme2', 'Fructose-6-phosphate [c]'), ('Enzyme26', '6-Phosphogluconolactone [c]')], 'Fructose-6-phosphate [c]': [('Enzyme2', 'Glucose-6-phosphate [c]'), ('Enzyme3', 'Fructose-1-6-phosphate [c]'), ('TALDO', 'Erythrose-4-phosphate [c]'), ('TKT', 'Erythrose-4-phosphate [c]'), ('TALDO', 'Seduheptulose-7-phosphate [c]'), ('TALDO', 'Glyceraldehyde 3-phosphate [c]'), ('TKT', 'Xylulose-5-phosphate [c]')], 'Fructose-1-6-phosphate [c]': [('Enzyme4', 'Dihydroxyacetone phosphate [c]'), ('Enzyme4', 'Glyceraldehyde 3-phosphate [c]')], 'Dihydroxyacetone phosphate [c]': [('Enzyme5', 'Glyceraldehyde 3-phosphate [c]')], 'Glyceraldehyde 3-phosphate [c]': [('Enzyme6', '1-3-Bisphosphoglycerate [c]'), ('TKT', 'Xylulose-5-phosphate [c]'), ('TKT', 'Ribose-5-phosphate [c]'), ('TKT', 'Seduheptulose-7-phosphate [c]'), ('TALDO', 'Seduheptulose-7-phosphate [c]'), ('TALDO', 'Fructose-6-phosphate

#### **Build each of the networks**

In [74]:
#enzymes we can target
data = {}
for i in range(len(df)):
    data[df.iloc[i]['Enzyme']] = (df.iloc[i]["Cancer RNA count"], df.iloc[i]["Healthy RNA count"])
#put all the rows of the column 'Enzyme' in a list

healthy_non_used_enzymes = []
for i in data:
    if data[i][1] == 0:
        healthy_non_used_enzymes.append(i)

print(healthy_non_used_enzymes)


# need to create network for healthy cells:

healthy_network = {}

for metabolite in network:
    for i in range(len(network[metabolite])):
        if network[metabolite][i][0] not in healthy_non_used_enzymes:
            if metabolite not in healthy_network:
                healthy_network[metabolite] = [network[metabolite][i]]
            else:
                healthy_network[metabolite].append(network[metabolite][i])

print(healthy_network)

# need to create network for cancer cells:

cancer_non_used_enzymes = []
for i in data:
    if data[i][0] == 0:
        cancer_non_used_enzymes.append(i)

print(cancer_non_used_enzymes)


# need to create network for cancer cells:

cancer_network = {}

for metabolite in network:
    for i in range(len(network[metabolite])):
        if network[metabolite][i][0] not in cancer_non_used_enzymes:
            if metabolite not in cancer_network:
                cancer_network[metabolite] = [network[metabolite][i]]
            else:
                cancer_network[metabolite].append(network[metabolite][i])

print(cancer_network)




['Enzyme12', 'Enzyme22', 'Enzyme24', 'Enzyme25', 'Enzyme26']
{'Glucose [c]': [('Enzyme1', 'Glucose-6-phosphate [c]'), ('Enzyme33', 'ADP-Glucose [c]')], 'Glucose-6-phosphate [c]': [('Enzyme2', 'Fructose-6-phosphate [c]')], 'Fructose-6-phosphate [c]': [('Enzyme2', 'Glucose-6-phosphate [c]'), ('Enzyme3', 'Fructose-1-6-phosphate [c]'), ('TALDO', 'Erythrose-4-phosphate [c]'), ('TKT', 'Erythrose-4-phosphate [c]'), ('TALDO', 'Seduheptulose-7-phosphate [c]'), ('TALDO', 'Glyceraldehyde 3-phosphate [c]'), ('TKT', 'Xylulose-5-phosphate [c]')], 'Fructose-1-6-phosphate [c]': [('Enzyme4', 'Dihydroxyacetone phosphate [c]'), ('Enzyme4', 'Glyceraldehyde 3-phosphate [c]')], 'Dihydroxyacetone phosphate [c]': [('Enzyme5', 'Glyceraldehyde 3-phosphate [c]')], 'Glyceraldehyde 3-phosphate [c]': [('Enzyme6', '1-3-Bisphosphoglycerate [c]'), ('TKT', 'Xylulose-5-phosphate [c]'), ('TKT', 'Ribose-5-phosphate [c]'), ('TKT', 'Seduheptulose-7-phosphate [c]'), ('TALDO', 'Seduheptulose-7-phosphate [c]'), ('TALDO', 'Fruc

#### **Search Algorithm and other algorithms from previous question**

In [75]:
def find_paths(metabolite, path, paths, visited, new_network):
    visited = visited.copy()
    if (metabolite in visited) and (visited[metabolite]):
        return
    """ print("metabolite and visited", metabolite, visited) """
    if metabolite in endpoints:
        if metabolite not in paths:
            paths[metabolite] = [path]
        else:
            paths[metabolite].append(path)
    elif metabolite not in new_network:
        return
    else:
        if visited[metabolite]:
            return

            
        for enzyme, product in new_network[metabolite]:
            visited[metabolite] = True
            find_paths(product, path + [(enzyme, product)], paths, visited, new_network)

def find_non_essential_enzymes(paths):
    non_used_enzymes_for_endpoint = {}
    for endpoint in paths:
        #find intersection of all the enzymes used in the paths to the endpoint
        essential_enzymes = []
        for element in paths[endpoint][0]:
            essential_enzymes.append(element[0])

        for path in paths[endpoint]:
            new_essential_enzymes = []

            for element in path:
                new_essential_enzymes.append(element[0])

            essential_enzymes = list(set(essential_enzymes).intersection(set(new_essential_enzymes)))

        non_used_enzymes_for_endpoint[endpoint] = list(set(enzymes) - set(essential_enzymes))
    return non_used_enzymes_for_endpoint

def find_all_non_essential_enzymes(non_used_enzymes_for_endpoint):
    all_non_used_enzymes = []

    for endpoint in non_used_enzymes_for_endpoint:
        all_non_used_enzymes += non_used_enzymes_for_endpoint[endpoint]
    all_non_used_enzymes = list(set(all_non_used_enzymes))

    for endpoint in non_used_enzymes_for_endpoint:
        for e in all_non_used_enzymes:
            if e not in non_used_enzymes_for_endpoint[endpoint]:
                all_non_used_enzymes.remove(e)
    return all_non_used_enzymes

#### **Get essential and non-essential enzymes for eahc network**

In [76]:
healthy_paths = {}

healthy_visited = {}
for metabolite in healthy_network:
    healthy_visited[metabolite] = False

find_paths('Glucose [c]', [], healthy_paths, healthy_visited, healthy_network)

healthy_non_essential_enzymes_for_endpoint = find_non_essential_enzymes(healthy_paths)

healthy_non_essential_enzymes = find_all_non_essential_enzymes(healthy_non_essential_enzymes_for_endpoint)
healthy_essential_enzymes = sorted(list(set(enzymes) - set(healthy_non_essential_enzymes)))


print("all non essential enzymes for healthy cells: ", healthy_non_essential_enzymes)
print("all essential enzymes for healthy cells: ", healthy_essential_enzymes)

all non essential enzymes for healthy cells:  ['Enzyme14', 'Enzyme33', 'Enzyme26', 'Enzyme31', 'Enzyme21', 'Enzyme27', 'Enzyme34', 'Enzyme5', 'Enzyme7', 'Enzyme29', 'Enzyme18', 'Enzyme28', 'Enzyme22', 'Enzyme23', 'Enzyme35', 'Transporter2', 'Enzyme25', 'Enzyme30', 'Enzyme17', 'Enzyme3', 'Enzyme4', 'Enzyme20', 'Enzyme19', 'Transporter1', 'TALDO', 'Enzyme32', 'Enzyme24', 'Enzyme12']
all essential enzymes for healthy cells:  ['Enzyme1', 'Enzyme10', 'Enzyme11', 'Enzyme13', 'Enzyme15', 'Enzyme16', 'Enzyme2', 'Enzyme6', 'Enzyme8', 'Enzyme9', 'PseudoEnzymes', 'TKT']


In [77]:
cancer_paths = {}

cancer_visited = {}
for metabolite in cancer_network:
    cancer_visited[metabolite] = False

find_paths('Glucose [c]', [], cancer_paths, cancer_visited, cancer_network)

cancer_non_essential_enzymes_for_endpoint = find_non_essential_enzymes(cancer_paths)

cancer_non_essential_enzymes = find_all_non_essential_enzymes(cancer_non_essential_enzymes_for_endpoint)
cancer_essential_enzymes = sorted(list(set(enzymes) - set(cancer_non_essential_enzymes)))


print("all non essential enzymes for cancer cells: ", cancer_non_essential_enzymes)
print("all essential enzymes for cancer cells: ", cancer_essential_enzymes)

all non essential enzymes for cancer cells:  ['Enzyme14', 'Enzyme33', 'Enzyme26', 'Enzyme13', 'Enzyme31', 'Enzyme15', 'Enzyme27', 'Enzyme34', 'Enzyme5', 'Enzyme7', 'Enzyme29', 'Enzyme18', 'Enzyme28', 'Enzyme35', 'Enzyme25', 'Enzyme30', 'Enzyme17', 'Enzyme3', 'Enzyme4', 'Enzyme20', 'Enzyme19', 'TALDO', 'Enzyme2', 'Enzyme32', 'Enzyme11', 'Enzyme12']
all essential enzymes for cancer cells:  ['Enzyme1', 'Enzyme10', 'Enzyme16', 'Enzyme21', 'Enzyme22', 'Enzyme23', 'Enzyme24', 'Enzyme6', 'Enzyme8', 'Enzyme9', 'PseudoEnzymes', 'TKT', 'Transporter1', 'Transporter2']


#### **Find enzymes to target**

In [78]:
# find the intersection between the essential enzymes for cancer cells and the non essential enzymes for healthy cells

target_enzymes = list(set(healthy_non_essential_enzymes).intersection(set(cancer_essential_enzymes)))

print("target enzymes: ", target_enzymes)


target enzymes:  ['Enzyme21', 'Enzyme22', 'Transporter2', 'Transporter1', 'Enzyme24', 'Enzyme23']


**Enzymes to target are: Enzyme21, Enzyme22, Transporter2, Transporter1, Enzyme24, Enzyme23**